In [78]:
import duckdb
import pandas as pd
import plotly.express as px
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_colwidth', None)
con = duckdb.connect(database='../orchestration/data/coches.net.duckdb', read_only=True)
import warnings
warnings.filterwarnings("ignore")

def search_cars_by_keywords_in_title(keywords: list, 
                                     case_insensitive=True, 
                                     limit: int = None,
                                     **kwargs
    ) -> pd.DataFrame: 
    if not keywords:
        raise ValueError("You must provide a list of keywords with at least one element")
    condition = f"title {'i' if case_insensitive else ''}like"
    query = f"""
        SELECT * from analytics.stg_cars
        WHERE {condition}'%{keywords[0]}%'
    """    
    for keyword in keywords[1:]:
        query += f" AND {condition}'%{keyword}%'"
    query = add_query_filters(query, **kwargs)
    if limit:
        query += f" LIMIT {limit}"
    return con.execute(query).df()


def add_query_filters(query : str,
     min_price: int = None,
     max_price: int = None,
     min_year: int = None,
     max_year: int = None,
     min_km: int = None,
     max_km: int = None,
     fuel_type: str = None
):
    if min_price is not None:
        query += f" AND price >= {min_price}"
    if max_price is not None:
        query += f" AND price <= {max_price}"
    if min_year is not None:
        query += f" AND year >= {min_year}"
    if max_year is not None:
        query += f" AND year <= {max_year}"
    if min_km is not None:
        query += f" AND km >= {min_km}"
    if max_km is not None:
        query += f" AND km <= {max_km}"
    if fuel_type is not None:
        query += f" AND fuel_type = '{fuel_type}'"
    return query

def search_prices_by_keywords_in_title(keywords: list, case_insensitive=True, limit: int = None, **kwargs) -> pd.DataFrame: 
    if not keywords:
        raise ValueError("You must provide a list of keywords with at least one element")
    condition = f"title {'i' if case_insensitive else ''}like"
    query = f"""
        WITH joined as (
        
                SELECT stg.*, fct.price, fct.date_key
                FROM analytics.stg_cars_snapshots stg
                LEFT JOIN analytics.fct_car_prices fct ON
                    stg.scd_id = fct.scd_id
        )
        select * from joined
        WHERE {condition}'%{keywords[0]}%'
    """
    for keyword in keywords[1:]:
        query += f" AND {condition}'%{keyword}%'"
    query = add_query_filters(query, **kwargs)
    if limit:
        query += f" LIMIT {limit}"
    return con.execute(query).df()


def plot_cars_search(keywords, **kwargs):
    df = search_cars_by_keywords_in_title(keywords=keywords, **kwargs)
    return px.scatter(df, x='price', y='km', color='fuel_type', trendline="ols", marginal_x="box",
                     title=f"Keywords: {' '.join(keywords)}")

def plot_prices_search(keywords, **kwargs):
    df = search_prices_by_keywords_in_title(keywords=keywords, **kwargs)
    return px.line(df, x='date_key', y='price', color='car_id',
                     title=f"Keywords: {' '.join(keywords)}")

con.execute('SELECT MAX(creationDate) FROM coches').df()

,"max(""creationDate"")"
0,2022-07-14T14:00:41Z


In [87]:
plot_cars_search(["HYUNDAI", "I30", "FASTBACK", 'KLASS'])

In [86]:
plot_cars_search(["HYUNDAI", "I30", "FASTBACK", 'TECNO'])

In [88]:
plot_prices_search(["HYUNDAI", "I30", "FASTBACK", 'KLASS'], fuel_type='Híbrido', max_km=10, max_price=22000)

In [92]:
plot_prices_search(["HYUNDAI", "I30", "FASTBACK", 'TECNO'], fuel_type='Híbrido', max_km=5000, max_price=23000)

## Ventas y cambios de precio
- 2022-06-29: HYUNDAI i30 1.0 TGDI 48V Klass Fastback 5p - 10km - Murcia - Vendido a 22440€
- 2022-06-29: HYUNDAI i30 1.0 TGDI 48V Tecno Fastback 5p. - 6550km - Jaén - Vendido a 25990€
- 2022-06-30: HYUNDAI i30 1.0 TGDI 48V Klass Fastback 5p. - 1475km - Valencia- Vendido a 22500€
